# **NAME: PARTHA SAKHA PAUL**

# **ROLL: MA23M016**

    CS6910_assignment 2


In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import numpy as np

# data_dir = '/content/drive/My Drive/nature_12K'
data_dir = 'inaturalist_12K'


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# data_dir = '/content/drive/My Drive/nature_12K/inaturalist_12K'


In [30]:
# Defining transforms for the training, validation, and testing sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(data_dir + '/train', transform=transform)
print(len(train_data))
# print(train_data[9998])
# Creating data indices for training and validation splits:
dataset_size = len(train_data)  #9999
indices = list(range(dataset_size)) #[0,1,...,9998]
split = int(np.floor(0.2 * dataset_size))  #1999
print(split)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
# print((val_indices))

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
# print(type(train_sampler))

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=train_sampler)
# print(len(train_loader))
validation_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=valid_sampler)
# print(*validation_loader)

9999
1999


In [33]:
activation_functions = {
            'relu': F.relu,
            'gelu': F.gelu,
            'silu': F.silu,
            'mish': F.mish
        }

# defining a class CNN that inherits from nn.Module which is PyTorch's base class for all neural network
class my_CNN(nn.Module):
    def __init__(self, num_classes=10, num_filters=[32, 64, 128, 256, 512], filter_sizes=[3, 3, 3, 3, 3], activation_fn='relu', num_neurons_dense=512):
        super(my_CNN, self).__init__()
        # Initializing class variables
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.activation_fn = activation_functions[activation_fn]
        self.num_neurons_dense = num_neurons_dense

        # Creating convolution layers using ModuleList
        self.conv_layers = nn.ModuleList()
        in_channels = 3  # since, input images are RGB
        for out_channels, kernel_size in zip(num_filters, filter_sizes):
            # a convolutional layer with in-out channels and kernel size
            conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, padding=0)
            self.conv_layers.append(conv_layer)
            # Update in_channels for the next layer
            in_channels = out_channels

        # Placeholder for dense layer will be initialized after calculating the flatten size
        self.dense = None
        # output layer: maps from the dense layer to the number of classes.
        self.out = nn.Linear(num_neurons_dense, num_classes)
        # find the size needed to flatten the conv layer outputs, initializing the dense layer accordingly
        self.init_flatten_size(input_shape=(3, 224, 224))  # input images are 224x224 RGB images
        # initializing the dense layer from flatten size
        self.dense = nn.Linear(self.flatten_size, num_neurons_dense)


    def init_flatten_size(self, input_shape):
        # to disable gradient calculations for speed up this computation
        with torch.no_grad():
            # a dummy input tensor of the correct shape
            input_tensor = torch.zeros(1, *input_shape)
            # Forward propagate through the conv layers to find output size
            output = self.forward_conv_layers(input_tensor)
            # total number of output features is the flat size needed for the dense layer input
            self.flatten_size = output.numel()

    # a function for sequentially passing input through all convolutional layers and applying activation and pooling
    def forward_conv_layers(self, x):
        # Convolution blocks
        for conv in self.conv_layers:
            x = conv(x)  # Convolution operation
            x = self.activation_fn(x)  # Apply the specified activation function
            x = F.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2
        return x

    # forward method defines how the input tensor is transformed through the model
    def forward(self, x):
        # Passing input through the convolution blocks
        x = self.forward_conv_layers(x)
        # Flattening the output for the dense layer
        x = torch.flatten(x, 1)   # Flatten the output for the dense layer
        # Dense layer
        x = self.dense(x)
        x = self.activation_fn(x)
        # Output layer
        x = self.out(x)
        return x

    # Method for training and evaluating the model
    def train_and_evaluate(self, train_loader, validation_loader, n_epochs=10, lr=0.001, device=None):
        if device is None:
            # setting up CUDA if available, otherwise, using CPU
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)  # transferring the model to the selected device (GPU or CPU)

        # Initializing the loss function and optimizer
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)

        # Training loop
        for epoch in range(n_epochs):
            self.train()  # Setting model to training mode
            training_loss = 0.0

            # Iterate over the training data
            for images, labels in train_loader:
                # transferring images and labels to the current device (GPU or CPU)
                images, labels = images.to(device), labels.to(device)
                # clearing the gradients of all optimized variables
                optimizer.zero_grad()
                # Forward prop: computing predicted outputs by passing inputs to the model
                outputs = self(images)
                # calculating the loss between predicted and true labels
                loss = loss_function(outputs, labels)
                # Backward prop: computing gradient of the loss with respect to parameters
                loss.backward()
                # optimization step (parameter update)
                optimizer.step()
                # updating running training loss
                training_loss += loss.item() * images.size(0)
            
            # average loss over an epoch
            training_loss /= len(train_loader.sampler)

            # Evaluation phase
            self.eval()  # Setting model to evaluation mode
            # initializing the validation loss for the epoch
            validation_loss = 0.0
            # tracking number of correctly predicted instances
            correct = 0
            # tracking total number of instances
            total = 0
            # disable gradient calculation for validation, saving memory and computations
            with torch.no_grad():
                # iterating over the validation data loader
                for images, labels in validation_loader:
                    # transferring images and labels to the current device (GPU or CPU)
                    images, labels = images.to(device), labels.to(device)
                    # Forward prop: computing predicted outputs by passing inputs to the model
                    outputs = self(images)
                    # calculating the loss between predicted and true labels
                    loss = loss_function(outputs, labels)
                    # updating running validation loss
                    validation_loss += loss.item() * images.size(0)
                    # converting output probabilities to predicted class
                    _, predicted = torch.max(outputs.data, 1)
                    # updating total number of instances
                    total += labels.size(0)
                    # updating correctly predicted instances
                    correct += (predicted == labels).sum().item()

            # average validation loss over an epoch
            validation_loss /= len(validation_loader.sampler)
            # validation accuracy
            validation_accuracy = correct / total

            # printing training and validation results
            print(f'Epoch {epoch+1}, Training Loss: {training_loss:.4f}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}')
#             wandb.log({"accuracy": validation_accuracy, "loss": validation_loss})


# model with parameters
model = my_CNN(num_classes=10,
                  num_filters=[32, 64, 128, 256, 512],
                  filter_sizes=[3, 3, 3, 3, 3],
                  activation_fn=F.relu,
                  num_neurons_dense=512)

print(model)

model.train_and_evaluate(train_loader, validation_loader, n_epochs=10, lr=0.001)

my_CNN(
  (conv_layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  )
  (out): Linear(in_features=512, out_features=10, bias=True)
  (dense): Linear(in_features=12800, out_features=512, bias=True)
)
Epoch 1, Training Loss: 2.2935, Validation Loss: 2.2495, Validation Accuracy: 0.1626
Epoch 2, Training Loss: 2.1979, Validation Loss: 2.1496, Validation Accuracy: 0.2161
Epoch 3, Training Loss: 2.1071, Validation Loss: 2.0998, Validation Accuracy: 0.2461
Epoch 4, Training Loss: 2.0572, Validation Loss: 2.0554, Validation Accuracy: 0.2541
Epoch 5, Training Loss: 2.0091, Validation Loss: 2.0114, Validation Accuracy: 0.2846
Epoch 6, Training Loss: 1.9459, Validation Loss: 1.9966, Validation Accuracy: 0.2736
Epoch 7, Training Los

In [24]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

key = input('Enter your API:')
wandb.login(key=key)  #25c2257eaf6c22aa056893db14da4ee2bf0a531a

Enter your API:25c2257eaf6c22aa056893db14da4ee2bf0a531a


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [25]:
sweep_config = {
    'method': 'bayes',  # can be grid, random
    'name' : 'cnn1',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
            'values': [[32, 64, 128, 256, 512], [64, 64, 64, 64, 64]] 
        },
        'activation_fn': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
#         'dropout': {
#             'values': [0.2, 0.3]
#         },
    }
}

sweep_id = wandb.sweep(sweep_config, project="Deep_leraning_A2")

Create sweep with ID: oypngjeo
Sweep URL: https://wandb.ai/parthasakhapaul/Deep_leraning_A2/sweeps/oypngjeo


In [34]:
def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        run_name = "-ac_"+wandb.config.activation_fn+"-filters_"+str(wandb.config.num_filters)
        wandb.run.name=run_name
        # Model training
        model = my_CNN(num_classes=10,
                  num_filters=wandb.config.num_filters,
                  filter_sizes=[3, 3, 3, 3, 3],
                  activation_fn=wandb.config.activation_fn,
                  num_neurons_dense=512)
        model.train_and_evaluate(train_loader, validation_loader, n_epochs=10, lr=0.001)        

wandb.agent(sweep_id, function=main, count=20)  # to run 20 experiments
wandb.finish()